In [18]:
import re
from collections import defaultdict
import numpy as np

def read_xml_to_string(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# 示例文件路径，实际应用中请替换为实际文件路径
file_path = 'conf/behaviors.xml'
xml_content = read_xml_to_string(file_path)
xml_content

'\ufeff<?xml version="1.0" encoding="UTF-8" ?>\n<Mascot xmlns="http://www.group-finity.com/Mascot" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"\n\txsi:schemaLocation="http://www.group-finity.com/Mascot Mascot.xsd">\n\n\t<BehaviorList>\n\t\n\t\t<!-- ALWAYS REQUIRED -->\n\t\t<Behavior Name="ChaseMouse" Frequency="0" Hidden="true">\n\t\t\t<NextBehavior Add="false">\n\t\t\t\t<BehaviorReference Name="SitAndFaceMouse" Frequency="1" />\n\t\t\t</NextBehavior>\n\t\t</Behavior>\n\t\t\n\t\t<Behavior Name="SitAndFaceMouse" Frequency="0">\n\t\t\t<NextBehavior Add="false">\n\t\t\t\t<BehaviorReference Name="SitAndFaceMouse" Frequency="100" />\n\t\t\t\t<BehaviorReference Name="SitAndSpinHead" Frequency="1" />\n\t\t\t\t<BehaviorReference Name="SitWhileDanglingLegs" Frequency="1" />\n\t\t\t</NextBehavior>\n\t\t</Behavior>\n\t\t\n\t\t<Behavior Name="SitAndSpinHead" Frequency="0">\n\t\t\t<NextBehavior Add="false">\n\t\t\t\t<BehaviorReference Name="SitAndFaceMouse" Frequency="1" />\n\t\t\t</NextBe

In [15]:
# 提取行为和下一个行为的关系
behavior_pattern = r'<Behavior Name="([^"]+)"[^>]*>[\s\S]*?<NextBehavior[^>]*>([\s\S]*?)</NextBehavior>'
next_behavior_pattern = r'<BehaviorReference Name="([^"]+)" Frequency="(\d+)"'

# 存储行为及其可能的下一个行为和频率
behavior_dict = defaultdict(dict)

# 解析XML文本
for behavior_match in re.finditer(behavior_pattern, xml_content):
    behavior_name = behavior_match.group(1)
    next_behaviors_content = behavior_match.group(2)
    for next_behavior_match in re.finditer(next_behavior_pattern, next_behaviors_content):
        next_behavior_name = next_behavior_match.group(1)
        frequency = int(next_behavior_match.group(2))
        behavior_dict[behavior_name][next_behavior_name] = frequency

# 确保包括所有唯一的行为名称
all_behaviors = set(behavior_dict.keys())
for behaviors in behavior_dict.values():
    all_behaviors.update(behaviors.keys())


In [16]:
behavior_dict

defaultdict(dict,
            {'ChaseMouse': {'SitAndFaceMouse': 1},
             'SitAndFaceMouse': {'SitAndFaceMouse': 100,
              'SitAndSpinHead': 1,
              'SitWhileDanglingLegs': 1},
             'SitAndSpinHead': {'SitAndFaceMouse': 1},
             'Fall': {'SitWhileDanglingLegs': 100, 'LieDown': 100},
             'SitWhileDanglingLegs': {'SitDown': 100,
              'CrawlAlongIECeiling': 100,
              'CrawlAlongWorkAreaFloor': 100},
             'SplitIntoTwo': {'LieDown': 1},
             'WalkLeftAlongFloorAndSit': {'LieDown': 1}})

In [17]:
all_behaviors

{'ChaseMouse',
 'CrawlAlongIECeiling',
 'CrawlAlongWorkAreaFloor',
 'Fall',
 'LieDown',
 'SitAndFaceMouse',
 'SitAndSpinHead',
 'SitDown',
 'SitWhileDanglingLegs',
 'SplitIntoTwo',
 'WalkLeftAlongFloorAndSit'}

In [ ]:

# 构建Markov Matrix
states = list(all_behaviors)
matrix_size = len(states)
markov_matrix = np.zeros((matrix_size, matrix_size), dtype=int)

# 填充矩阵
for i, behavior in enumerate(states):
    transitions = behavior_dict.get(behavior, {})
    for next_behavior, frequency in transitions.items():
        if next_behavior in states:
            j = states.index(next_behavior)
            markov_matrix[i, j] = frequency

# 归一化
markov_matrix_n = markov_matrix / markov_matrix.sum(axis=1, keepdims=True)

# 打印结果
print("States:", states)
print("Markov Matrix:\n", markov_matrix)
print("Markov Matrix Normalized:\n", markov_matrix_n)

States: ['SitAndFaceMouse', 'SitWhileDanglingLegs', 'SitAndSpinHead', 'ChaseMouse']
Markov Matrix:
 [[100   1   1   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  1   0   0   0]]
Markov Matrix Normalized:
 [[0.98039216 0.00980392 0.00980392 0.        ]
 [       nan        nan        nan        nan]
 [       nan        nan        nan        nan]
 [1.         0.         0.         0.        ]]


C:\Users\18193\AppData\Local\Temp\ipykernel_18684\4141249111.py:15: RuntimeWarning: invalid value encountered in true_divide
  markov_matrix_n = markov_matrix / markov_matrix.sum(axis=1, keepdims=True)
